In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
import numpy as np
import pandas as pd

In [7]:
#Initialize dummy matrices
card_wins_matrix = np.zeros((8,8))
trick_wins_matrix = np.zeros((8,8))

#Fill dummy matrices
i = 0
for x in range(8):
    for z in range(8):
        card_wins_matrix[x,z] = i
        trick_wins_matrix[x,z] = 64-i
        i+=1

print(card_wins_matrix)
print(trick_wins_matrix)
#Save dummy matrices
np.save("card_wins.npy",card_wins_matrix)
np.save("trick_wins.npy",trick_wins_matrix)

[[ 0.  1.  2.  3.  4.  5.  6.  7.]
 [ 8.  9. 10. 11. 12. 13. 14. 15.]
 [16. 17. 18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29. 30. 31.]
 [32. 33. 34. 35. 36. 37. 38. 39.]
 [40. 41. 42. 43. 44. 45. 46. 47.]
 [48. 49. 50. 51. 52. 53. 54. 55.]
 [56. 57. 58. 59. 60. 61. 62. 63.]]
[[64. 63. 62. 61. 60. 59. 58. 57.]
 [56. 55. 54. 53. 52. 51. 50. 49.]
 [48. 47. 46. 45. 44. 43. 42. 41.]
 [40. 39. 38. 37. 36. 35. 34. 33.]
 [32. 31. 30. 29. 28. 27. 26. 25.]
 [24. 23. 22. 21. 20. 19. 18. 17.]
 [16. 15. 14. 13. 12. 11. 10.  9.]
 [ 8.  7.  6.  5.  4.  3.  2.  1.]]


In [6]:
%run simulation.ipynb

Winning probabilities for each sequence combination (Player 1 vs Player 2):

Player 1: BBB vs Player 2: BBB --> Probability that Player 1 wins: 0.99800
Player 1: BBB vs Player 2: BBR --> Probability that Player 1 wins: 0.39500
Player 1: BBB vs Player 2: BRB --> Probability that Player 1 wins: 0.19500
Player 1: BBB vs Player 2: BRR --> Probability that Player 1 wins: 0.11200
Player 1: BBB vs Player 2: RBB --> Probability that Player 1 wins: 0.00200
Player 1: BBB vs Player 2: RBR --> Probability that Player 1 wins: 0.16500
Player 1: BBB vs Player 2: RRB --> Probability that Player 1 wins: 0.01200
Player 1: BBB vs Player 2: RRR --> Probability that Player 1 wins: 0.31100
Player 1: BBR vs Player 2: BBB --> Probability that Player 1 wins: 0.44700
Player 1: BBR vs Player 2: BBR --> Probability that Player 1 wins: 1.00000
Player 1: BBR vs Player 2: BRB --> Probability that Player 1 wins: 0.80100
Player 1: BBR vs Player 2: BRR --> Probability that Player 1 wins: 0.88500
Player 1: BBR vs Player

In [14]:
win_probabilities1, deck_history1, sequences1 = calculate_win_probabilities_with_decks(rounds=1000, use_card_accumulation=False)

In [15]:
win_probabilities2, deck_history2, sequences2 = calculate_win_probabilities_with_decks(rounds=1000, use_card_accumulation=True)

In [18]:
def matrix_creator(win_probs, deck_hist, seq):

    combinations = ['BBB', 'BBR', 'BRB', 'BRR', 'RBB', 'RBR', 'RRB', 'RRR']

    matches = []

    matrix = pd.DataFrame(0.0, index=combinations, columns=combinations)
    
    for i in range(len(seq)):
        for j in range(len(seq)):
            p1 = binary_to_string(seq[i])
            p2 = binary_to_string(seq[j])
            prob = win_probs[i, j]
            matrix.loc[p1, p2] = float(prob)

    return matrix

matrix1 = matrix_creator(win_probabilities1, deck_history1, sequences1)
matrix2 = matrix_creator(win_probabilities2, deck_history2, sequences2)

matrix2

,BBB,BBR,BRB,BRR,RBB,RBR,RRB,RRR
BBB,0.999,0.483,0.312,0.209,0.000,0.294,0.010,0.486
BBR,0.491,1.000,0.867,0.961,0.000,0.936,0.474,0.987
BRB,0.669,0.116,1.000,0.461,0.359,0.513,0.041,0.648
BRR,0.764,0.029,0.496,1.000,0.489,0.586,0.997,1.000
RBB,1.000,1.000,0.611,0.454,1.000,0.512,0.024,0.761
RBR,0.678,0.046,0.463,0.390,0.456,1.000,0.114,0.671
RRB,0.985,0.469,0.940,0.003,0.965,0.872,1.000,0.512
RRR,0.472,0.010,0.310,0.000,0.202,0.302,0.464,0.999


In [9]:
sequences = ['BBB', 'BBR', 'BRB', 'BRR', 'RBB', 'RBR', 'RRB', 'RRR']
def create_heatmaps(card_wins = 'card_wins.npy', trick_wins = 'trick_wins.npy'):
    
    matrix1 = matrix_creator(win_probabilities1, deck_history1, sequences1)
    matrix2 = matrix_creator(win_probabilities2, deck_history2, sequences2)
   
    card_wins_matrix = np.load(card_wins)
    trick_wins_matrix = np.load(trick_wins)

    card_wins_df = pd.DataFrame(card_wins_matrix, index = sequences, columns = sequences)
    trick_wins_df = pd.DataFrame(trick_wins_matrix, index = sequences, columns = sequences)

    #Heatmap for number of cards
    plt.figure(figsize=(10,8))
    sns.heatmap(card_wins_df,
                annot=True,
                cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256),
                linewidths=.5,
                cbar_kws={'label': 'Win Probability'})
    plt.title('Win Probabilities for Card Wins')
    plt.xlabel('Player 2 Choice')
    plt.ylabel('Player 1 Choice')
    plt.savefig('figures/p2_card_probs.png', bbox_inches = 'tight')

    #Heatmap for number of tricks
    plt.figure(figsize=(10,8))
    sns.heatmap(trick_wins_df,
                annot=True,
                cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256),
                linewidths=.5,
                cbar_kws={'label': 'Win Probability'})
    plt.title('Win Probabilities for Trick Wins')
    plt.xlabel('Player 2 Choice')
    plt.ylabel('Player 1 Choice')
    plt.savefig('figures/p2_trick_probs.png', bbox_inches = 'tight')

In [ ]:
create_heatmaps()